In [0]:
# Customers who have ordered more than one item
from pyspark.sql import functions as F

# read tables from raw_zone
customers = spark.table("workspace.raw_zone.customer_2020")
orders = spark.table("workspace.raw_zone.orders_2020")

# Join, aggregate, and apply HAVING filter
result_df = (
    customers.alias("c")
    .join(
        orders.alias("o"),
        F.col("c.CustomerKey") == F.col("o.CustomerKey"),
        "inner"
    )
    .groupBy(
        F.col("c.CustomerKey"),
        F.col("c.Firstname"),
        F.col("c.Lastname")
    )
    .agg(
        F.sum(F.col("o.OrderQuantity")).alias("TotalOrderQuantity")
    )
    .filter(F.col("TotalOrderQuantity") > 1)  
)

result_df.display()

result_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.transformation_layer.curated_TotalOrderQuantity")